In [1]:
import gspread as gs
import gspread_dataframe as gd
import pandas as pd
import random
import string
from datetime import datetime, timedelta

In [2]:
def generate_first_names_last_names(num_names):
    first_names = []
    last_names = []
    
    while num_names:
        # Generating a random first name with length between 1 and 10
        first_name_length = random.randint(1, 10)
        first_name = ''.join(random.choices(string.ascii_lowercase, k=first_name_length)).capitalize()
        
        # Generating a random last name with length between 1 and 10
        last_name_length = random.randint(1, 10)
        last_name = ''.join(random.choices(string.ascii_lowercase, k=last_name_length)).capitalize()

        if first_name in first_names or first_name in last_names:
            continue
        
        if last_name in first_names or last_name in last_names:
            continue
        
        num_names -= 1

        # Combining first and last names
        first_names.append(first_name)
        last_names.append(last_name)

    return first_names, last_names

In [3]:
# Make timestamp dummy

def generate_timestamp(num_rows, max_hours_ago=8):
    """let's say we got this data from 8 hours ago"""
    time_format = r"%d/%m/%Y %H:%M:%S"

    timestamp = [(datetime.now() - timedelta(weeks=random.randint(0,156),days=random.randint(0,5), hours=random.randint(7,max_hours_ago+7), 
                                minutes=random.randint(0,60),
                                seconds= random.randint(0, 60))).strftime(time_format) 
                                for _ in range(num_rows)]
    
    return timestamp

In [4]:
input = r"./input/bank-marketing-dataset - bank-marketing-dataset (1).csv"

In [5]:
bank_marketing = pd.read_csv(input)
bank_marketing_df = bank_marketing.copy()

In [6]:
num_rows, num_columns = bank_marketing_df.shape 

In [7]:
bank_marketing_df.head()

,age,job,marital,education,default,housing,loan
0,56,housemaid,married,basic.4y,no,no,no
1,57,services,married,high.school,unknown,no,no
2,37,services,married,high.school,no,yes,no
3,40,admin.,married,basic.6y,no,no,no
4,56,services,married,high.school,no,no,yes


In [8]:
bank_marketing_df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan'], dtype='object')

In [16]:
bank_marketing_df.isin(["unknown"]).sum()

age             0
job           330
marital        80
education    1731
default      8597
housing       990
loan          990
dtype: int64

In [31]:
bank_marketing_df["default"].value_counts()

default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

In [37]:
bank_marketing_df[bank_marketing_df["default"] != "unknown"].isin(["unknown"]).sum()

age             0
job           178
marital        69
education    1183
default         0
housing       763
loan          763
dtype: int64

: 

In [34]:
bank_marketing_df[["education","job", "default"]].groupby("default").value_counts().sort_values()

default  education            job        
yes      high.school          unemployed        1
unknown  illiterate           retired           1
                              admin.            1
                              housemaid         1
         university.degree    unknown           2
                                             ... 
no       high.school          services       2078
         basic.9y             blue-collar    2546
         professional.course  technician     2836
         high.school          admin.         2966
         university.degree    admin.         5126
Name: count, Length: 177, dtype: int64

In [20]:
bank_marketing_df

,age,job,marital,education,default,housing,loan
0,56,housemaid,married,basic.4y,no,no,no
1,57,services,married,high.school,unknown,no,no
2,37,services,married,high.school,no,yes,no
3,40,admin.,married,basic.6y,no,no,no
4,56,services,married,high.school,no,no,yes
...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no
41184,46,blue-collar,married,professional.course,no,no,no
41185,56,retired,married,university.degree,no,yes,no
41186,44,technician,married,professional.course,no,no,no


In [160]:
bank_marketing_df[['job', 'marital', 'education', 'default', 'housing', 'loan']] = bank_marketing_df[['job', 'marital', 'education', 'default', 'housing', 'loan']].astype('category')

In [161]:
bank_marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   age        41188 non-null  int64   
 1   job        41188 non-null  category
 2   marital    41188 non-null  category
 3   education  41188 non-null  category
 4   default    41188 non-null  category
 5   housing    41188 non-null  category
 6   loan       41188 non-null  category
dtypes: category(6), int64(1)
memory usage: 564.6 KB


In [162]:
#Generate timestamp
bank_marketing_df["timestamp"] = generate_timestamp(num_rows= num_rows, max_hours_ago= 8)
bank_marketing_df["timestamp"] = pd.to_datetime(bank_marketing_df["timestamp"], format= r"%d/%m/%Y %H:%M:%S")

#Generate first name and last name
bank_marketing_df["first_name"], bank_marketing_df["last_name"] = generate_first_names_last_names(num_rows)

#Generate Email
bank_marketing_df["email"] = [first_name.lower()+last_name.lower()+"@gmail.com" for first_name, last_name in zip(bank_marketing_df["first_name"], bank_marketing_df["last_name"])]

# add birth of date from age
bank_marketing_df["birth_of_date"] = bank_marketing_df["age"].apply(lambda x: (datetime.now() - timedelta(weeks=x*52,days= random.randint(1, 30))).strftime(r"%d/%m/%Y"))

In [170]:
bank_marketing_df.nunique()

Timestamp        41154
Email            41188
First Name       41188
Last Name        41188
Birth of Date     1788
Job                 12
Marital              4
Education            8
Default              3
Housing              3
Loan                 3
dtype: int64

In [173]:
bank_marketing_df['Education'].value_counts()

Education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

In [172]:
bank_marketing_df['Marital'].value_counts()

Marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

In [163]:
bank_marketing_df[['timestamp', 'first_name', 'last_name', 'email', 'birth_of_date', 'job', 'marital', 'education', 'default', 'housing', 'loan']].to_csv(r'./input/transformed2.csv',index=False)

In [164]:
bank_marketing_df = bank_marketing_df[['timestamp', 'email', 'first_name', 'last_name', 'birth_of_date', 'job', 'marital', 'education', 'default', 'housing', 'loan']]
bank_marketing_df.head()

,timestamp,email,first_name,last_name,birth_of_date,job,marital,education,default,housing,loan
0,2020-11-16 10:32:59,nnnbtdtcofdxq@gmail.com,Nnnbtd,Tcofdxq,07/01/1968,housemaid,married,basic.4y,no,no,no
1,2022-02-21 14:00:35,ttwuvinhndtkqeo@gmail.com,Ttwuv,Inhndtkqeo,01/01/1967,services,married,high.school,unknown,no,no
2,2021-12-24 09:10:02,rusbiutywawi@gmail.com,Rusbiutyw,Awi,07/12/1986,services,married,high.school,no,yes,no
3,2022-03-10 12:48:32,dtjbevzbildqr@gmail.com,Dtjbevzbi,Ldqr,15/12/1983,admin.,married,basic.6y,no,no,no
4,2022-03-17 09:44:42,yroeweedpurlob@gmail.com,Yroeweedpu,Rlob,24/12/1967,services,married,high.school,no,no,yes


In [165]:
bank_marketing_df.columns = ["Timestamp", "Email", "First Name", "Last Name", "Birth of Date", "Job", "Marital", "Education", "Default", "Housing", "Loan"]

In [166]:
bank_marketing_df_sorted = bank_marketing_df.sort_values(by='Timestamp', ascending=False)
bank_marketing_df_sorted

,Timestamp,Email,First Name,Last Name,Birth of Date,Job,Marital,Education,Default,Housing,Loan
1966,2023-11-13 14:46:37,ygrcooluhdjcxjvvi@gmail.com,Ygrcoolu,Hdjcxjvvi,31/12/1969,admin.,married,basic.4y,unknown,no,no
13094,2023-11-13 14:22:55,crccrtedcrnqffpxyx@gmail.com,Crccrtedcr,Nqffpxyx,17/12/1991,blue-collar,single,basic.4y,unknown,no,no
28916,2023-11-13 14:06:37,toldhrdzeddvus@gmail.com,Toldhrdzed,Dvus,21/01/1967,blue-collar,married,basic.9y,no,no,no
36490,2023-11-13 14:03:59,vjvbvlbtxaia@gmail.com,Vjvbvl,Btxaia,05/12/1986,self-employed,married,professional.course,no,no,no
8793,2023-11-13 13:51:59,fohsijnxmeqec@gmail.com,Fohsijn,Xmeqec,24/11/1994,services,single,high.school,no,yes,no
...,...,...,...,...,...,...,...,...,...,...,...
32389,2020-11-11 06:16:55,gtajivhrkyoejn@gmail.com,Gtajivh,Rkyoejn,06/12/1992,management,married,university.degree,no,no,no
22814,2020-11-11 06:12:42,pmhhycwynwbc@gmail.com,Pmh,Hycwynwbc,31/12/1964,admin.,divorced,university.degree,unknown,yes,no
21840,2020-11-11 06:05:36,esoropfdabfczrwkd@gmail.com,Esoropfdab,Fczrwkd,14/12/1983,technician,single,professional.course,no,no,no
24420,2020-11-11 05:59:24,bqqpxtydzlnecvi@gmail.com,Bqqpxty,Dzlnecvi,04/01/1975,management,married,university.degree,no,no,no


In [167]:
gc = gs.service_account(filename='df11-group5-bc883e1b40d0.json')
sheet = "https://docs.google.com/spreadsheets/d/1DLIiR135fsZOb6Kt-AI_QnajUzjmgl_rk2xo097554w/edit#gid=1794896340"
ws = gc.open_by_url(sheet).worksheet("Form responses 2")
existing = gd.get_as_dataframe(ws)
ws.clear()
gd.set_with_dataframe(worksheet=ws, dataframe=bank_marketing_df_sorted, include_index=False, resize=True)
